# Using pandas with large data

Based on https://www.dataquest.io/blog/pandas-big-data/

DataFrameSummary https://github.com/mouradmourafiq/pandas-summary

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display

In [3]:
import pandas as pd
from pandas_summary import DataFrameSummary
import numpy as np

PATH='data/'

In [4]:
print(pd.__version__)

0.23.4


In [5]:
table_names = ['holidays_events', 'items', 'oil', 'stores', 'transactions',
               'test', 'train']

## Numpy Subtype Ranges

In [55]:
int_types = ["uint8", "uint16", "uint32", "uint64"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for uint16
---------------------------------------------------------------
min = 0
max = 65535
---------------------------------------------------------------

Machine parameters for uint32
---------------------------------------------------------------
min = 0
max = 4294967295
---------------------------------------------------------------

Machine parameters for uint64
---------------------------------------------------------------
min = 0
max = 18446744073709551615
---------------------------------------------------------------



In [54]:
int_types = ["int8", "int16", "int32", "int64"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------



In [74]:
float_types = ["float16", "float32", "float64"]
for ft in float_types:
    print(np.finfo(ft))

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
---------------------------------------------------------------

Machine parameters for float32
---------------------------------------------------------------
precision =   6   resolution = 1.0000000e-06
machep =    -23   eps =        1.1920929e-07
negep =     -24   epsneg =     5.9604645e-08
minexp =   -126   tiny =       1.1754944e-38
maxexp =    128   max =        3.4028235e+38
nexp =        8   min =        -max
---------------------------------------------------------------

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   e

## `train.csv`

### Info

In [6]:
train = pd.read_csv(f'{PATH}{table_names[6]}.csv')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [12]:
DataFrameSummary(train).summary()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.25497e+08,NaN,1.25497e+08,1.25497e+08,1.25497e+08,NaN
mean,6.27485e+07,NaN,27.4646,972769,8.55487,NaN
std,3.62279e+07,NaN,16.3305,520534,23.6052,NaN
min,0,NaN,1,96995,-15372,NaN
25%,3.13743e+07,NaN,12,522383,2,NaN
50%,6.27485e+07,NaN,28,959500,4,NaN
75%,9.41228e+07,NaN,43,1.35438e+06,9,NaN
max,1.25497e+08,NaN,54,2.12711e+06,89440,NaN
counts,125497040,125497040,125497040,125497040,125497040,103839389
uniques,125497040,1684,54,4036,258474,2


### Memory Consumption

In [18]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 6 columns):
id             int64
date           object
store_nbr      int64
item_nbr       int64
unit_sales     float64
onpromotion    object
dtypes: float64(1), int64(3), object(2)
memory usage: 15.3 GB


In [9]:
def avg_mem_usage_columns(df):
    for dtype in ['float','int','object']:
        selected_dtype = df.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

In [21]:
avg_mem_usage_columns(train)

Average memory usage for float columns: 478.73 MB
Average memory usage for int columns: 718.10 MB
Average memory usage for object columns: 3959.48 MB


### Optimizing Numeric Columns with Subtypes

In [8]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [39]:
train_int = train.select_dtypes(include=['int'])
converted_int = train_int.apply(pd.to_numeric,downcast='unsigned')

Memory Usage
Before: 2872.40 MB
After: 1077.15 MB
Number of columns


,before,after
uint8,NaN,1.0
uint32,NaN,2.0
int64,3.0,NaN


In [40]:
print('Memory Usage')
print(f'Before: {mem_usage(train_int)}')
print(f'After: {mem_usage(converted_int)}')

print('\nNumber of columns')
compare_ints = pd.concat([train_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

Memory Usage
Before: 2872.40 MB
After: 1077.15 MB

Number of columns


,before,after
uint8,NaN,1.0
uint32,NaN,2.0
int64,3.0,NaN


In [41]:
train_float = train.select_dtypes(include=['float'])
converted_float = train_float.apply(pd.to_numeric,downcast='float')

In [43]:
print('Memory Usage')
print(f'Before: {mem_usage(train_float)}')
print(f'After: {mem_usage(converted_float)}')

print('\nNumber of columns')
compare_floats = pd.concat([train_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

Memory Usage
Before: 957.47 MB
After: 478.73 MB

Number of columns


,before,after
float32,NaN,1.0
float64,1.0,NaN


In [44]:
optimized_train = train.copy()

optimized_train[converted_int.columns] = converted_int
optimized_train[converted_float.columns] = converted_float

In [45]:
print('Memory Usage')
print(f'Before: {mem_usage(train)}')
print(f'After: {mem_usage(optimized_train)}')

Memory Usage
Before: 15708.31 MB
After: 13434.33 MB


### Optimizing Object Types Using Categoricals

**For `store_nbr` column**

In [46]:
store_nbr = train.store_nbr
store_nbr_cat = store_nbr.astype('category')

In [52]:
print('Memory Usage for store_nbr')
print(f'Before: {mem_usage(store_nbr)}')
print(f'After: {mem_usage(store_nbr_cat)}')

Memory Usage for store_nbr
Before: 957.47 MB
After: 119.69 MB


**Convert each object column to category if the number of unique values is less than 50%**

In [51]:
train_obj = train.select_dtypes(include=['object']).copy()

converted_obj = pd.DataFrame()

for col in train_obj.columns:
    num_unique_values = len(train_obj[col].unique())
    num_total_values = len(train_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = train_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = train_obj[col]

KeyError: MemoryError()

In [ ]:
print('Memory Usage')
print(f'Before: {mem_usage(train_obj)}')
print(f'After: {mem_usage(converted_obj)}')

print('\nNumber of columns')
compare_obj = pd.concat([train_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

# Optimizing `train.csv`

In [53]:
train.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'], dtype='object')

In [6]:
column_types = {'id': 'uint32',
                'store_nbr': 'category',
                'item_nbr': 'category',
                'unit_sales': 'category',
                'onpromotion': 'category'}

train_opt = pd.read_csv(f'{PATH}{table_names[6]}.csv', dtype=column_types, parse_dates=['date'], infer_datetime_format=True)

In [10]:
print('Memory Usage')
# print(f'Before: {mem_usage(train)}')
print(f'After: {mem_usage(train_opt)}')

Memory Usage
After: 2419.70 MB


In [11]:
train_opt.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 6 columns):
id             uint32
date           datetime64[ns]
store_nbr      category
item_nbr       category
unit_sales     category
onpromotion    category
dtypes: category(4), datetime64[ns](1), uint32(1)
memory usage: 2.4 GB


In [66]:
def mem_usage(df):
    sizes = list(df.memory_usage(deep=True) / 1024 ** 2)[1:]
    types = [t.name for t in df.dtypes]
    cols = list(df.columns)
    
    mem = pd.DataFrame({'columns': cols, 'size': sizes, 'type': types})
    
    return mem

mem_usage(train_opt)

,columns,size,type
0,id,478.733215,uint32
1,date,957.466431,datetime64[ns]
2,store_nbr,119.688844,category
3,item_nbr,239.767878,category
4,unit_sales,504.360354,category
5,onpromotion,119.683497,category


**Col `date` as a categorical**

In [69]:
column_types = {'id': 'uint32',
                'date': 'category',
                'store_nbr': 'category',
                'item_nbr': 'category',
                'unit_sales': 'category',
                'onpromotion': 'category'}

train_cat_all = pd.read_csv(f'{PATH}{table_names[6]}.csv', dtype=column_types)

In [72]:
train_cat_all.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 6 columns):
id             uint32
date           category
store_nbr      category
item_nbr       category
unit_sales     category
onpromotion    category
dtypes: category(5), uint32(1)
memory usage: 1.7 GB


In [70]:
mem_usage(train_cat_all)

,columns,size,type
0,id,478.733215,uint32
1,date,239.552334,category
2,store_nbr,119.688844,category
3,item_nbr,239.767878,category
4,unit_sales,504.360354,category
5,onpromotion,119.683497,category
